In [53]:
import numpy as np
import pyscf
import pyscf.gto
import pyscf.qmmm
import pyscf.scf

mol = pyscf.gto.M(atom='N 0 0 0.; N 0 0 1.', basis='631G', verbose=3)
deltaZ = np.array((-0.00, 0))
includeonly = [0, 1]

calc = pyscf.scf.RHF(mol)


mf = pyscf.qmmm.mm_charge(calc, mol.atom_coords()/(1 / 0.52917721067), deltaZ)
class NoSelfQMMM(mf.__class__):
        def energy_nuc(self):
                return self.mol.energy_nuc()
mf = NoSelfQMMM()
mf.kernel()
mycc = pyscf.cc.CCSD(mf).run()
print ('Energy:', mycc.energy() + mf.kernel()+deltaZ[0]*(7+deltaZ[1])/(1/0.52917721067) + deltaZ[1]*(7+deltaZ[0])/(1/0.52917721067))
print ('expected:', -110.8722842)
dm1 = mycc.make_rdm1()
dm1_ao = np.einsum('pi,ij,qj->pq', mf.mo_coeff, dm1, mf.mo_coeff.conj())

for site in includeonly:
    # method A
    #q = mol._atm.copy()
    #q[site, 0] = 0
    #a = pyscf.gto.getints('int1e_nuc_sph', q, mol._bas, mol._env)
    #print ('EPN, method A', np.matmul(a, dm1_ao).trace() / mol.atom_charges()[site])
    
    # method B
    mol.set_rinv_orig_(mol.atom_coords()[site])
    print ('EPN, method B', np.matmul(mol.intor('int1e_rinv'), dm1_ao).trace())

converged SCF energy = -108.835236570774
E(CCSD) = -109.041857111477  E_corr = -0.2066205407024847
converged SCF energy = -108.835236570774
Energy: -109.04185711147682
expected: -110.8722842
EPN, method B 21.964057317822466
EPN, method B 21.964057317822515


In [32]:
mol.atom_coords()/(1 / 0.52917721067)

array([[0., 0., 0.],
       [0., 0., 1.]])

In [15]:
0.05*(7.05)/(1/0.52917721067)

0.18653496676117504

In [28]:
mol.atom_coords()/(1 / 0.52917721067)

array([[0., 0., 0.],
       [0., 0., 1.]])

In [46]:
dm1_ao.diagonal()

array([2.08510789, 0.50211401, 1.05217241, 0.42821792, 0.41463665,
       0.6641638 , 0.11165596, 0.17542803, 0.07905939, 2.08510789,
       0.50211401, 1.05217241, 0.42821792, 0.41463665, 0.6641638 ,
       0.11165596, 0.17542803, 0.07905939])

In [47]:
dm1.diagonal()

array([1.99993837, 1.99994299, 1.98899603, 1.97588856, 1.9476586 ,
       1.9476586 , 1.98092735, 0.04386803, 0.04386803, 0.00602777,
       0.00400162, 0.0059458 , 0.0059458 , 0.01215502, 0.00940992,
       0.00940992, 0.01007105, 0.00828655])

In [48]:
mol.intor('int1e_rinv')

array([[ 5.29177211e-01,  1.17591325e-01,  9.09491487e-02,
         0.00000000e+00,  0.00000000e+00,  2.30300834e-02,
         0.00000000e+00,  0.00000000e+00,  3.37425082e-03,
         5.75757734e-05,  2.32692260e-02,  4.51926896e-02,
         0.00000000e+00,  0.00000000e+00, -4.43854682e-02,
         0.00000000e+00,  0.00000000e+00, -7.43013701e-02],
       [ 1.17591325e-01,  5.28554613e-01,  4.06756179e-01,
         0.00000000e+00,  0.00000000e+00,  1.59269386e-01,
         0.00000000e+00,  0.00000000e+00,  1.01031049e-01,
         1.25363842e-01,  2.60412985e-01,  2.84327331e-01,
         0.00000000e+00,  0.00000000e+00, -2.99326727e-01,
         0.00000000e+00,  0.00000000e+00, -3.06615213e-01],
       [ 9.09491487e-02,  4.06756179e-01,  4.85888913e-01,
         0.00000000e+00,  0.00000000e+00,  1.48099628e-01,
         0.00000000e+00,  0.00000000e+00,  1.86327523e-01,
         2.82410294e-01,  4.12498415e-01,  4.46315998e-01,
         0.00000000e+00,  0.00000000e+00, -1.94712635e